In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('flights_test.csv')

In [3]:
df['month'] = pd.DatetimeIndex(df.fl_date).month
df['year'] = pd.DatetimeIndex(df.fl_date).year
df['day'] = pd.DatetimeIndex(df.fl_date).day
df['dayofweek'] = pd.DatetimeIndex(df.fl_date).weekday
df['dayofyear'] = pd.DatetimeIndex(df.fl_date).dayofyear

In [4]:
df['month']=df['month'].apply(lambda x: '{0:0>2}'.format(x))
df['day']=df['day'].apply(lambda x: '{0:0>2}'.format(x))
df['dayofyear'] = df['dayofyear'].apply(lambda x: '{0:0>3}'.format(x))
df['crs_dep_time'] = df['crs_dep_time'].apply(lambda x: '{0:0>4}'.format(x))
df['crs_arr_time'] = df['crs_arr_time'].apply(lambda x: '{0:0>4}'.format(x))
df['date'] = (df.year.astype(str) + df.month.astype(str) + df.day.astype(str)).astype(int)

In [5]:
#Creating uid to match with average delay data
df['op_carrier_fl_num']=df['op_carrier_fl_num'].apply(lambda x: '{0:0>4}'.format(x))
df['uid'] = df.origin_airport_id.astype(str) + df.op_carrier_fl_num.astype(str) + df.dest_airport_id.astype(str)

In [6]:
#Importing csv with computation of average delay times
avg_delays = pd.read_csv('../SQL/average_delays2.csv')
avg_delays['op_carrier_fl_num']=avg_delays['op_carrier_fl_num'].apply(lambda x: '{0:0>4}'.format(x))

In [7]:
#Combining airport_id + op_carrier_fl_num to form one uid. Basis of merger
avg_delays['uid'] = avg_delays.oid.astype(str) + avg_delays.op_carrier_fl_num.astype(str) + avg_delays.did.astype(str) 

In [8]:
#Merging average delays to model dataset.
df = df.merge(avg_delays, how='left', on='uid')

In [9]:
df.op_unique_carrier = df.op_unique_carrier.replace(['WN','AA','OO','DL','UA','MQ','YX','OH','B6','9E',
'YV','AS','EV','NK','PT','F9','QX','ZW','AX','G7','G4','CP','HA','C5','VX','EM','KS','9K'], list((range(28))))

In [10]:
df = df.fillna(0)

In [11]:
X = df[['date', 'dayofweek', 'dayofyear', 'op_unique_carrier',
       'op_carrier_fl_num_x', 'origin_airport_id', 'dest_airport_id',
       'crs_dep_time','crs_arr_time', 'distance',
       'arr_del_avg', 'dep_del_avg', 'car_del_avg', 'wth_del_avg',
       'nas_del_avg', 'sec_del_avg', 'acft_del_avg', 'taxi_in_avg',
       'taxi_out_avg', 'elp_time_diff_avg']]

In [12]:
#Scaling the input dataset
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)

In [13]:
#To retrieve model from disk
import pickle
loaded_model = pickle.load(open('mid_termXGB.sav','rb'))

In [14]:
delays = loaded_model.predict(scaled_X)

In [15]:
df['predicted_delays'] = delays

In [16]:
final = df[['fl_date','mkt_carrier',
       'mkt_carrier_fl_num','origin','dest','predicted_delays']]

In [17]:
final.to_csv('josh_vishal_submission.csv', index =False)